        Date    Product  Starting Inventory      Demand  Order Quantity  \
0 2023-01-01  Product A                  70   64.914568              76   
1 2023-01-01  Product B                 147   46.543392               0   
2 2023-01-01  Product C                  94   48.982238               0   
3 2023-01-01  Product D                  90  103.929381               0   
4 2023-01-01  Product E                  67   36.199856               0   

   Sold Quantity  Ending Inventory  Holding Cost  Ordering Cost  Sales  \
0             52                94     83.846893      43.834626    520   
1             50                97    151.625913       0.000000   1000   
2             37                57     95.076391       0.000000    555   
3             59                31    101.715352       0.000000   1770   
4             41                26     88.258336       0.000000   1025   

   Stock Out  Lead Time Days  Lead Time Variability  Overstock  \
0   0.000000               7          

In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import dual_annealing

# Define parameters
product_cost = {'Product A': 10, 'Product B': 20, 'Product C': 15, 'Product D': 30, 'Product E': 25}
holding_cost_rate = 0.1  # 10% of product cost
ordering_cost = 50  # Fixed cost per order

# Load your dataset
def load_data(file_path):
    return pd.read_csv(file_path)

# Define stochastic simulation
def stochastic_simulation(df, num_simulations=1000):
    results = []
    for sim in range(num_simulations):
        for product in df['Product'].unique():
            product_data = df[df['Product'] == product]
            for _, row in product_data.iterrows():
                starting_inventory = row['Starting Inventory']
                base_demand = row['Demand']
                demand_variability = 10  # Standard deviation for demand variability

                # Simulate demand
                simulated_demand = np.random.normal(loc=base_demand, scale=demand_variability)
                simulated_demand = max(0, simulated_demand)

                # Calculate new ending inventory and costs
                sales = min(starting_inventory, simulated_demand)
                ending_inventory = starting_inventory - sales

                if ending_inventory < simulated_demand / 2:  # Reorder condition
                    order_qty = np.random.randint(50, 150)
                    ending_inventory += order_qty
                    ordering_cost_val = ordering_cost
                else:
                    order_qty = 0
                    ordering_cost_val = 0

                holding_cost_val = ending_inventory * holding_cost_rate * product_cost[product]

                results.append({
                    'Simulation': sim,
                    'Product': product,
                    'Date': row['Date'],
                    'Simulated Demand': simulated_demand,
                    'Ending Inventory': ending_inventory,
                    'Order Quantity': order_qty,
                    'Holding Cost': holding_cost_val,
                    'Ordering Cost': ordering_cost_val,
                    'Sales': sales * product_cost[product],
                    'Stock Out': 1 if sales < simulated_demand else 0
                })

    return pd.DataFrame(results)

# Define optimization model using Simulated Annealing
def optimize_inventory_simulated_annealing(simulation_df):
    optimized_results = []

    def objective(x, product_data):
        # x[0]: Order Quantity
        holding_cost = holding_cost_rate * product_cost[product_data['Product'].iloc[0]] * x[0]
        ordering_cost = ordering_cost if x[0] > 0 else 0
        stock_out_cost = sum(product_data['Simulated Demand'] - x[0] * product_data.shape[0])
        return holding_cost + ordering_cost + stock_out_cost

    bounds = [(50, 150)]  # Order quantity bounds

    for product in simulation_df['Product'].unique():
        product_data = simulation_df[simulation_df['Product'] == product]

        result = dual_annealing(objective, bounds, args=(product_data,))
        optimal_order_quantity = result.x[0]

        optimized_results.append({
            'Product': product,
            'Optimal Order Quantity': optimal_order_quantity,
            'Total Demand': product_data['Simulated Demand'].sum(),
            'Total Sales': product_data['Sales'].sum()
        })

    return pd.DataFrame(optimized_results)

# Generate next year's dataset based on optimization
def generate_next_year_data(optimized_df, start_date='2024-01-01', end_date='2025-01-01'):
    date_range = pd.date_range(start_date, end_date, freq='M')
    products = optimized_df['Product'].unique()
    next_year_data = {
        'Date': [],
        'Product': [],
        'Starting Inventory': [],
        'Demand': [],
        'Order Quantity': [],
        'Sold Quantity': [],
        'Ending Inventory': [],
        'Holding Cost': [],
        'Ordering Cost': [],
        'Sales': [],
        'Stock Out': []
    }

    for date in date_range:
        for product in products:
            row = optimized_df[optimized_df['Product'] == product].iloc[0]
            starting_inventory = row['Optimal Order Quantity']
            demand = np.random.normal(loc=row['Total Demand'], scale=10)
            demand = max(0, demand)
            sales = min(starting_inventory, demand)
            ending_inventory = starting_inventory - sales

            if ending_inventory < demand / 2:  # Reorder condition
                order_qty = np.random.randint(50, 150)
                ending_inventory += order_qty
                ordering_cost_val = ordering_cost
            else:
                order_qty = 0
                ordering_cost_val = 0

            holding_cost_val = ending_inventory * holding_cost_rate * product_cost[product]

            next_year_data['Date'].append(date)
            next_year_data['Product'].append(product)
            next_year_data['Starting Inventory'].append(starting_inventory)
            next_year_data['Demand'].append(demand)
            next_year_data['Order Quantity'].append(order_qty)
            next_year_data['Sold Quantity'].append(sales)
            next_year_data['Ending Inventory'].append(ending_inventory)
            next_year_data['Holding Cost'].append(holding_cost_val)
            next_year_data['Ordering Cost'].append(ordering_cost_val)
            next_year_data['Sales'].append(sales * product_cost[product])
            next_year_data['Stock Out'].append(1 if sales < demand else 0)

    return pd.DataFrame(next_year_data)

# Main function to execute the process
def main(input_path, output_path):
    df = load_data(input_path)
    simulation_df = stochastic_simulation(df)
    optimized_df = optimize_inventory_simulated_annealing(simulation_df)
    next_year_data = generate_next_year_data(optimized_df)
    next_year_data.to_csv(output_path, index=False)
    print(f"Next year's optimized dataset saved to '{output_path}'")

# Set paths (replace with your actual file paths)
input_path = 'C:\\Users\\js031\\Downloads\\modified_inventory_management_data.csv'
output_path = 'C:\\Users\\js031\\Downloads\\next_year_optimized_inventory_data.csv'

# Run main function
main(input_path, output_path)


UnboundLocalError: cannot access local variable 'ordering_cost' where it is not associated with a value

In [4]:
import pandas as pd
import numpy as np
from pyomo.environ import *
from pyomo.opt import SolverFactory

# Load the dataset
data_path = "C:\\Users\\js031\\Downloads\\modified_inventory_management_data.csv"
df = pd.read_csv(data_path)

# Parameters
products = df['Product'].unique()
dates = pd.to_datetime(df['Date']).dt.strftime('%Y-%m').unique()

# Create the Pyomo model
model = ConcreteModel()

# Sets
model.Products = Set(initialize=products)
model.Dates = Set(initialize=dates)

# Parameters
demand = df.set_index(['Product', df['Date']])['Demand'].to_dict()
holding_cost = df.set_index(['Product', df['Date']])['Holding Cost'].to_dict()
ordering_cost = df.set_index(['Product', df['Date']])['Ordering Cost'].to_dict()
stockout_cost = df.set_index(['Product', df['Date']])['Stock Out'].to_dict()

# Variables
model.OrderQty = Var(model.Products, model.Dates, domain=NonNegativeReals)
model.EndingInventory = Var(model.Products, model.Dates, domain=NonNegativeReals)

# Objective Function: Minimize total cost
def total_cost_rule(model):
    return sum(holding_cost[(p, d)] * model.EndingInventory[p, d] +
               ordering_cost[(p, d)] * model.OrderQty[p, d] +
               stockout_cost[(p, d)] for p in model.Products for d in model.Dates)
model.TotalCost = Objective(rule=total_cost_rule, sense=minimize)

# Constraints
def inventory_balance_rule(model, p, d):
    # Get the previous date
    prev_date_idx = max(0, list(model.Dates).index(d) - 1)
    prev_date = list(model.Dates)[prev_date_idx]
    
    if d == min(model.Dates):
        return model.EndingInventory[p, d] == 0 + model.OrderQty[p, d] - demand[(p, d)]
    else:
        return model.EndingInventory[p, d] == model.EndingInventory[p, prev_date] + model.OrderQty[p, d] - demand[(p, d)]
model.InventoryBalance = Constraint(model.Products, model.Dates, rule=inventory_balance_rule)

# Robustness constraint: add uncertainty
def robust_constraint_rule(model, p, d):
    uncertainty_factor = 0.1  # Adjust this factor for robustness
    return model.OrderQty[p, d] >= demand[(p, d)] * (1 - uncertainty_factor)
model.RobustConstraint = Constraint(model.Products, model.Dates, rule=robust_constraint_rule)

# Solve the model
solver = SolverFactory('glpk')
solver.solve(model)

# Extract results
results = pd.DataFrame([(p, d, model.OrderQty[p, d].value, model.EndingInventory[p, d].value)
                        for p in model.Products for d in model.Dates],
                       columns=['Product', 'Date', 'Order Quantity', 'Ending Inventory'])

# Calculate total optimization value
total_optimization_value = model.TotalCost()

print(f"Total Optimization Value: {total_optimization_value}")

# Generate next year's dataset based on the optimization results
def generate_next_year_data(results, start_date='2025-01-01', end_date='2025-12-31'):
    date_range = pd.date_range(start_date, end_date, freq='M')
    next_year_data = {
        'Date': [],
        'Product': [],
        'Starting Inventory': [],
        'Demand': [],
        'Order Quantity': [],
        'Sold Quantity': [],
        'Ending Inventory': [],
        'Holding Cost': [],
        'Ordering Cost': [],
        'Sales': [],
        'Stock Out': []
    }

    for date in date_range:
        for _, row in results.iterrows():
            product = row['Product']
            starting_inventory = row['Ending Inventory']
            # Simulate demand with some variability
            demand = np.random.normal(loc=starting_inventory, scale=10)
            demand = max(0, demand)
            sales = min(starting_inventory, demand)
            ending_inventory = starting_inventory - sales

            # Determine order quantity and costs
            if ending_inventory < demand / 2:  # Reorder condition
                order_qty = np.random.randint(50, 150)
                ending_inventory += order_qty
                ordering_cost_val = 50  # Fixed ordering cost
            else:
                order_qty = 0
                ordering_cost_val = 0

            holding_cost_val = ending_inventory * 0.1  # Adjust according to holding cost
            sales_val = sales * (100)  # Placeholder for unit sale price
            stock_out = 1 if sales < demand else 0

            next_year_data['Date'].append(date)
            next_year_data['Product'].append(product)
            next_year_data['Starting Inventory'].append(starting_inventory)
            next_year_data['Demand'].append(demand)
            next_year_data['Order Quantity'].append(order_qty)
            next_year_data['Sold Quantity'].append(sales)
            next_year_data['Ending Inventory'].append(ending_inventory)
            next_year_data['Holding Cost'].append(holding_cost_val)
            next_year_data['Ordering Cost'].append(ordering_cost_val)
            next_year_data['Sales'].append(sales_val)
            next_year_data['Stock Out'].append(stock_out)

    return pd.DataFrame(next_year_data)

# Generate the dataset
next_year_df = generate_next_year_data(results)

# Save the dataset
output_path = 'C:\\Users\\js031\\Downloads\\next_year_data.csv'
next_year_df.to_csv(output_path, index=False)
print(f"Next year's dataset saved to: {output_path}")


ERROR: Rule failed when generating expression for Objective TotalCost with
index None: KeyError: ('Product A', '2023-01')
ERROR: Constructing component 'TotalCost' from data=None failed:
        KeyError: ('Product A', '2023-01')


KeyError: ('Product A', '2023-01')

In [9]:
import pandas as pd
from pyomo.environ import *
from pyomo.opt import SolverFactory
from datetime import timedelta

# Load the dataset
data_path = 'C:\\Users\\js031\\Downloads\\modified_inventory_management_data.csv'
df = pd.read_csv(data_path)

# Convert Date to datetime
df['Date'] = pd.to_datetime(df['Date'])

# Parameters
products = df['Product'].unique()
dates = df['Date'].dt.strftime('%Y-%m').unique()

# Create the Pyomo model
model = ConcreteModel()

# Sets
model.Products = Set(initialize=products)
model.Dates = Set(initialize=dates)

# Parameters
demand = df.set_index(['Product', df['Date'].dt.strftime('%Y-%m')])['Demand'].to_dict()
holding_cost = df.set_index(['Product', df['Date'].dt.strftime('%Y-%m')])['Holding Cost'].to_dict()
ordering_cost = df.set_index(['Product', df['Date'].dt.strftime('%Y-%m')])['Ordering Cost'].to_dict()
stockout_cost = df.set_index(['Product', df['Date'].dt.strftime('%Y-%m')])['Stock Out'].to_dict()

# Variables
model.OrderQty = Var(model.Products, model.Dates, domain=NonNegativeReals)
model.EndingInventory = Var(model.Products, model.Dates, domain=NonNegativeReals)

# Objective Function: Minimize total cost
def total_cost_rule(model):
    return sum(holding_cost[(p, d)] * model.EndingInventory[p, d] +
               ordering_cost[(p, d)] * model.OrderQty[p, d] +
               stockout_cost[(p, d)] for p in model.Products for d in model.Dates)
model.TotalCost = Objective(rule=total_cost_rule, sense=minimize)

# Constraints
def inventory_balance_rule(model, p, d):
    prev_date_idx = max(0, list(model.Dates).index(d) - 1)
    prev_date = list(model.Dates)[prev_date_idx]
    
    if d == min(model.Dates):
        return model.EndingInventory[p, d] == model.OrderQty[p, d] - demand[(p, d)]
    else:
        return model.EndingInventory[p, d] == model.EndingInventory[p, prev_date] + model.OrderQty[p, d] - demand[(p, d)]
model.InventoryBalance = Constraint(model.Products, model.Dates, rule=inventory_balance_rule)

# Robustness constraint: add uncertainty
def robust_constraint_rule(model, p, d):
    uncertainty_factor = 0.1  # Adjust this factor for robustness
    return model.OrderQty[p, d] >= demand[(p, d)] * (1 - uncertainty_factor)
model.RobustConstraint = Constraint(model.Products, model.Dates, rule=robust_constraint_rule)

# Solve the model
solver = SolverFactory('glpk')
solver.solve(model)

# Extract results
results = pd.DataFrame([(p, d, model.OrderQty[p, d].value, model.EndingInventory[p, d].value)
                        for p in model.Products for d in model.Dates],
                       columns=['Product', 'Date', 'Order Quantity', 'Ending Inventory'])

# Extend to the next year
next_year_dates = pd.date_range(start=df['Date'].max() + timedelta(days=1), periods=12, freq='M').strftime('%Y-%m')

next_year_data = []
for p in products:
    last_inventory = results[(results['Product'] == p) & (results['Date'] == dates[-1])]['Ending Inventory'].values[0]
    for d in next_year_dates:
        # Here, we'll assume demand, costs, and other factors remain the same as the last year for simplicity
        demand_next = demand.get((p, dates[-1]), 0)
        hold_cost = holding_cost.get((p, dates[-1]), 0)
        order_cost = ordering_cost.get((p, dates[-1]), 0)
        stockout_cost_val = stockout_cost.get((p, dates[-1]), 0)
        
        order_qty = demand_next * (1 - 0.1)  # Using the robustness constraint for the next year
        ending_inventory = max(0, last_inventory + order_qty - demand_next)
        
        next_year_data.append([p, d, demand_next, hold_cost, order_cost, stockout_cost_val, order_qty, ending_inventory])
        last_inventory = ending_inventory

next_year_df = pd.DataFrame(next_year_data, columns=['Product', 'Date', 'Demand', 'Holding Cost', 'Ordering Cost', 'Stock Out', 'Order Quantity', 'Ending Inventory'])

# Save the results
results.to_csv('C:\\Users\\js031\Downloads\\next_year_data.csv', index=False)
next_year_df.to_csv('C:\\Users\\js031\Downloads\\next_year_data.csv', index=False)

print("Optimization results and next year's data saved.")


Optimization results and next year's data saved.
